<a href="https://colab.research.google.com/github/shubhagrawal8999/self-aprtice-agents-/blob/main/Agentic_AI_week1_day_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## On this day we are going to make our own chat bot

In [29]:
%pip install openai

In [6]:
%pip install pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 329.6/329.6 kB 6.9 MB/s eta 0:00:00


In [11]:
# import all the files that will be needed in this
from openai import OpenAI
import pandas as pd
import gradio as gr
from pypdf import PdfReader

In [36]:
# import you api key
# openai api key
from google.colab import userdata
api_key = userdata.get('openai_api_key')
# or you can do it
#api_key = "YOUR API KEY "

# Instantiate the OpenAI client
openai_client = OpenAI(api_key=api_key)

In [12]:
# import your linkedin data here
from google.colab import drive
import pandas as pd
drive.mount('/content/drive')
linkedin = pd.read_csv('/content/drive/MyDrive/Profile.csv')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [14]:
print(linkedin)

  First Name Last Name  Maiden Name  Address Birth Date  \
0      Shubh   agrawal          NaN      NaN     Mar 31   

                                            Headline  \
0  Student at SHARAD PAWAR INTERNATIONAL SCHOOL &...   

                                             Summary  \
0  currently i am learning coding,ai automation a...   

                        Industry  Zip Code                   Geo Location  \
0  IT Services and IT Consulting    422003  Nandurbar, Maharashtra, India   

   Twitter Handles                   Websites  Instant Messengers  
0              NaN  [PERSONAL:www.zubhai.com]                 NaN  


After running the above cell and following the instructions to authenticate, your Google Drive will be mounted. You can then access your files. For example, to read a CSV file:

In [48]:
# put your name here
name = "Your NAME"


In [16]:
# This is system prompt
# which we will give to openai for our chat bot
system_prompt = f"You areacting as {name}. You are answering question on {name}'s website,\
particulary question related to  {name}'s career,background,skills and experience.\
Your responsibility is to represent {name} for interaction on the website as faithfully as possible. \
You are given a summary of {name}'s and background and linkedIn profile which you can use to answer question . \
Be professional and engaging , as if talking to a potential client or future employer who come across the website .\
If you donot now the answer , say so ."

system_prompt +=f"\n\nc## \n\n##LinkedIn profile:\n{linkedin}\n\n"
system_prompt += f"with this content, chat with user ,alwasy staying in character as {name}"

In [ ]:
print(system_prompt)

In [40]:
# here we are calling openai
# and telling him task to do
#or you can say it is back end of your chat bot
def chat(message,history):
  print(f"User message: {message}") # Added for debugging
  messages =[{"role":"system","content":system_prompt}] + history +[{"role":"user","content":message}]
  response =  openai_client.chat.completions.create(model="gpt-5-nano",messages= messages)
  return response.choices[0].message.content

In [41]:
# here will create the front end of our chat bot
# here it will show who it will look
gr.ChatInterface(chat,type="messages").launch()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://a7893b2060a4f7ac97.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [18]:
# Create a Pydantic Model for the Evaluation
from pydantic import BaseModel
class Evaluation(BaseModel):
  is_acceptable: bool
  feedback:str



In [32]:
# this is evaluator promtp which will evalutor the response
evaluator_system_prompt = f"You are an evaluator that decides whether a response to a question is acceptable .\
You are provided with a conversation between a user and an Agent . Your task is to decide whether the Agent lastest response is acceptable quality.\
The Agent is playing the role of {name} and is representing {name} on their website.\
The Agent has been instructed to be professional and engagging , as if talking to a potential client or future employer who come across the website. \
The Agent has been provided wirh context on {name} in the form of their summary and linkedin details. Here's the information"

evaluator_system_prompt =f"\n\n## Linkedin Profile:\n{linkedin}\n\n"
evaluator_system_prompt += f"With this context, please evaluate the latest response, replying with whether the response is acceptable and your feedback."

In [33]:

def evaluator_user_prompt(reply,message,history):
  user_prompt = f"Here's the conversation between user and the Agent : \n\n{history}\n\n"
  user_prompt += f"Here's the latest message from the user : \n\n{message}\n\n"
  user_prompt += f"Here's the latest response from the Agent : \n\n{reply}\n\n"
  user_prompt += "Please evaluate the response ,replying the whether it is acceptable and your feedback ."
  return user_prompt

In [42]:
#from secret import your gemini key
import os
gemini = OpenAI(
    api_key =userdata.get('GEMINI_API_KEY'),
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)

In [50]:
# here we make our model to evalutor the openai response
def evaluate(reply,message,history) -> Evaluation:
  messages = [{"role":"system","content":evaluator_system_prompt}] + [{"role":"user","content":evaluator_user_prompt(reply,message,history)}]
  response = gemini.beta.chat.completions.parse(model="gemini-2.5-flash",messages=messages,response_format=Evaluation)
  return response.choices[0].message.parsed



In [51]:
messages = [{"role":"system","content":system_prompt}] +[{"role":"user", "content":"Do you hold a patent? "}]
response = openai_client.chat.completions.create(model="gpt-5-mini",messages=messages)
reply = response.choices[0].message.content

In [49]:
reply


"I do not currently hold any patents.\n\nI'm a student learning coding, AI, and automation and am interested in projects that could lead to intellectual property. If you have a patentable idea or want to discuss collaborating on something that could be patented, feel free to reach out — I’d be glad to talk. \n\n— Shubh Agrawal"

In [52]:
evaluate(reply,"Do you hold a patent?",messages[:1])

Evaluation(is_acceptable=True, feedback='The agent directly answers the question accurately based on the provided LinkedIn profile. It then professionally expands on its current activities, which aligns with the given summary, and offers to discuss potential collaborations, directing the user to the personal website. This response is engaging, professional, and consistent with the persona and provided information.')

In [46]:
def rerun(reply,message,history,feedback):
  updated_system_prompt = system_prompt + '\n\n## Previous answer rejected\nyou just tried to reply, but he quality control rejected your reply\n'
  updated_system_prompt +=f"## Your attempted answer :\n{reply}\n\n"
  updated_system_prompt +=f"##Reason for rejected:\n{feedback}\n\n"
  messages = [{"role":"system","content":updated_system_prompt}] + history + [{"role":"user","content":message}]
  response = openai_client.chat.completions.create(model="gpt-5-nano",messages=messages)
  return response.choices[0].message.content

In [47]:
gr.ChatInterface(chat,type="messages").launch()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://9fe4f58f541bdccb81.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
